<html>
<head>
	<title></title>
</head>
<body>
<h1 style="text-align: center;line-height: 120%;">ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ НАЦИОНАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ УНИВЕРСИТЕТ «МОСКОВСКИЙ ЭНЕРГЕТИЧЕСКИЙ ИНСТИТУТ» Кафедра «РЗиАЭ»</h1>
<h2 style="text-align: center;line-height: 20%;">Лабораторная работа №3</h2>

<h3 style="text-align: center;line-height: 80%;"><em>Генетический алгоритм</em></h3>

<p style="text-align: right;line-height: 40%;">Выполнил: студент группы Э-13м-19</p>
<p style="text-align: right;line-height: 40%;">Максимов Р.С.</p>

<p style="text-align: right;line-height: 40%;">Принял: Грачева Н. П.</p>
<h4 style="text-align: center;line-height: 100%;"><em> Москва, 2020</em></h3>
</body>
</html>

In [2]:
import numpy as np
import time
import random 
import scipy
from scipy import spatial
import math

In [774]:
def how_much_time(func):
    def decor_time(*args):
        start_time = time.time()
        result = func(*args)
        end_time = time.time()
        print("All time for searching= ",end_time - start_time)
        return end_time - start_time,result
    return decor_time

In [761]:
def create_new_population(size_word,alph):
    random_word = ''
    for i in range(size_word):
        random_index = random.randint(0,len(alph)-1)
        random_word = random_word + alph[random_index]
    return random_word

In [762]:
def generate_start_population(size,word,alphabet):
    list_start_population = [create_new_population(len(word),alphabet) for i in range(size)]
    return list_start_population    

In [763]:
def create_dict_for_word(word):
    dict_word = {}
    for i in range(len(word)):
        dict_word[i] = word[i]        
    return dict_word

In [764]:
def create_matric_for_search_cos_distance(population,dict_word):
    matric = np.zeros((len(population), len(dict_word)))
    number = 0
    for one_from_populations in population:         
        for i in range(0,len(one_from_populations)):
            one_letter = one_from_populations[i] 
            a = 0                
#             value = value_list[]
            if one_letter == dict_word[i]:
                a = a + 1            
                matric[number,i] = a    
        number = number + 1
    return matric

In [765]:
def crossover_between_parents(one_parent,two_parent,parents,alphabet):
    ra = random.randint(0,len(alphabet)-1)
    child=""
    mass_one_parent = parents[one_parent][1]
    mass_two_parent = parents[two_parent][1]
    for i in range(len(one_parent)):
        actual_info_letter_one = mass_one_parent[i]
        actual_info_letter_two = mass_two_parent[i]
        if actual_info_letter_one == 1 or actual_info_letter_two == 1:
            if (actual_info_letter_one == 1):
                child = child + one_parent[i]            
            elif actual_info_letter_two == 1:
                child = child + two_parent[i]
        else:            
            child = child + alphabet[ra]
    return child

In [766]:
def roulette_for_getting_parent(parents):
    r = random.randint(0,len(parents)-1)
    keys = list(parents.keys())
    random_parent = keys[r]
    cos_dist_parent = parents[random_parent][0] #расстояние вытаскиваем из списка
    return random_parent

In [767]:
def generate_new_population(size,parents,alphabet,word):
    i = 0
    actual_parents = []
    numbers_for_roulette = round(size - size/4)
    numbers_new = round(size - numbers_for_roulette)

    while i < numbers_for_roulette:
        one_parent = roulette_for_getting_parent(parents)
        two_parent = roulette_for_getting_parent(parents)
        child = crossover_between_parents(one_parent,two_parent,parents,alphabet)
        actual_parents.append(child)
        i = i + 1

    j = 0
    while j < numbers_new:
        actual_parents.append(create_new_population(len(word),alphabet))
        j = j + 1 
    return actual_parents

In [768]:
def splitting_word(words):
    word_list = words.split()
    new_word_list = []
    probel_list = []
#     print(word_list)
    for one_word in word_list:
        if (len(one_word) > 6):
            one_part = one_word[:round(len(one_word)/2)]
            new_word_list.append(one_part)
            probel_list.append('')
            two_part = one_word[round(len(one_word)/2):]
            new_word_list.append(two_part)
            probel_list.append(' ')
        else:
            new_word_list.append(one_word)
            probel_list.append(' ')
    return new_word_list,probel_list

In [843]:
@how_much_time
def genetic_algorithm(word, alphabet):
#     print('Hello')
    alphabet = alphabet.lower()
    set_alphabet = set()
    for i in range(len(alphabet)):
        set_alphabet.add(alphabet[i])
    
    alphabet = list(set_alphabet)
#     print(alphabet)
    numbers = 10
    new_word = ''
    word_list = word.split()
#     print(word_list)    
#     word_split = splitting_word(word)
#     word_list = word_split[0]
#     probel_list = word_split[1]
#     print(word_split)
    for one_from in word_list:
        flag = False
        if (one_from[0].isupper()):
#             print(one_from).
            flag = True
        one_from = one_from.lower()
        condition = 1.
        list_for_calc = [1] * len(one_from)
        parent_population = {}
        start_population = generate_start_population(numbers, one_from,alphabet)
        dict_for_word = create_dict_for_word(one_from)
        start_flag = True    
        while start_flag:
            matric = create_matric_for_search_cos_distance(start_population,dict_for_word)
            for i in range(0,matric.shape[0]):
                rass = scipy.spatial.distance.cosine(np.array(list_for_calc),matric[i,:])
                if not(math.isnan(rass)):
                    if condition < 0.1:
                        condition = 0.7
                    parent_population[start_population[i]] = [rass,matric[i,:]]
                    if(rass < condition):
                        condition = condition - 0.0001                        
                    else:
                        try:
                            parent_population.pop(start_population[i])
                        except KeyError:
                            continue

            if (len(parent_population) == 0):
                start_population = generate_start_population(numbers, one_from,alphabet)
            else:
                start_population = generate_new_population(numbers,parent_population,alphabet,one_from)
            if one_from in start_population:
                index = start_population.index(one_from)                
#                 new_word = new_word + probel_list[word_list.index(one_from)-1] + start_population[index]
                actual_new_word = start_population[index]
                if flag:
                    new_word = new_word + actual_new_word.title() + ' '
                else:
                    new_word = new_word + actual_new_word + ' '
#             
#                 print(new_word)
#                 print(parent_population)
    #             print(start_population.index(word))
    #             print(start_population[index])
    #             print(start_population)
                
                start_flag = False
                
    return new_word
        

In [844]:
@how_much_time
def full_search(word,alp):
    new_word = ''
    for i in range(len(word)):
        j = 0
        while j < len(alp):
            if(word[i] == alp[j]):
                new_word = new_word + alp[j]
                j = len(alp)
            else:
                j = j + 1
    
    return new_word

In [848]:
# # word = "роба"
# word = "блЮЗ"
# alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
# alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
word = "Хорошо ли? Натурально ли? Не преувеличил ли? — трепетал про себя Раскольников. — Зачем сказал: «женщины»?"
alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ-.,!:;?«»—"
# word = "Познакомились на концерте Little Big"
# alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
m = genetic_algorithm(word, alphabet)
print(m)
# n = full_search(word, alphabet)

All time for searching=  0.2592658996582031
(0.2592658996582031, 'Хорошо ли? Натурально ли? Не преувеличил ли? — трепетал про себя Раскольников. — Зачем сказал: «женщины»? ')


Вывод: после добавления функционала учета верхнего и нижнего регистра, время работы алгоритма снизилось в 3 раза